# <u>**Modélisation statistique de la valeur des joueurs de football à partir de leurs performances.**</u>

Par Roman Dupraz-Bardou, Lenny Ouakil et Paul Lemoine Vandermoere

### <u>**1.Introduction :**</u>

L’évaluation de la valeur des joueurs est une composante clé dans le football moderne. Des plateformes comme Transfermarkt et le CIES fournissent des estimations, mais les clubs cherchent souvent à obtenir des informations plus précises basées sur des performances actualisées. Ce projet vise à construire un modèle qui estime la valeur des joueurs à partir de diverses sources de données sur les performances (open data, scraping, API), puis à comparer les estimations avec celles des plateformes publiques comme Transfermarkt ou le CIES. Pour cela, nous nous concentrerons sur les sept grands championnats européens (Premier League, Ligue 1, Bundesliga, Liga, Serie A, Eredivisie, Liga Portugal) de 2015 à 2020 (championnats écourtés pour cause de pandémie du covid19).

Ainsi, nous nous attendons à observer une corrélation positive entre les performances des joueurs (nombre de buts, de passes décisives, taux de cleen sheet...) et leur valeur. Afin d'obtenir un nombre conséquent de données nécessaires à la construction de ce modèle, nous utiliserons des API comme API Football qui offrent des statistiques détaillées sur les performances des joueurs des sept grands championnats européens (xG, passes clés, interceptions, dribbles, etc.), du scraping depuis des plateformes comme Transfermarkt pour récupérer les valeurs marchandes publiques des joueurs. Il pourrait aussi être intéressant d'intégrer des variables démographiques et/ou caractéristiques (âge, position, durée du contrat) depuis des sources telles que Transfermarkt ou d'autres bases de données disponibles en open data. 

### <u> Sommaire</u>

1. [Installation](#installation)
2. [Obtention des données](#obtention-des-données)
   - [Scraping des données de performances depuis FBref](#scraping-depuis-fbref)
   - [Scraping des données de valeur marchande](#scraping-des-données-de-valeur-marchande)
   - [Utilisation d'API](#utilisation-d'api)
3. [Statistiques descriptives](#statistiques-descriptives)
4. [Modèle de prédiction](#modèle-de-prédiction)
5. [Conclusion](#conclusion)